# Tableau and player form

Here we create a csv file that is used for making further Tableau visualizations. 

In [1]:
# recent gameweek
gameweek = 22

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch player stats data
all_df = pd.read_csv('../data/fpl/data_week1.csv', index_col=0)
all_df['gameweek'] = 1
all_df['xPoints'] = all_df['adjusted points']

for week in range(2,gameweek+1):
    filepath = '../data/fpl/data_week' + str(week) + '.csv'
    df = pd.read_csv(filepath, index_col=0)
    df['gameweek'] = week
    column_name = 'xPoints week ' + str(week)
    df['xPoints'] = df[column_name]
    all_df = all_df.append(df,sort='False')

all_df.shape

(13303, 242)

In [4]:
# calculate player form
for i in [5,10]:
    gameweeks = np.arange(gameweek-i+1,gameweek+1)
    column_name = 'form ' + str(i)
    all_df.loc[all_df['gameweek']==gameweek, column_name] = (all_df[all_df['gameweek'].isin(gameweeks)].groupby(['id'])['xPoints'].sum() / (all_df.loc[(all_df['gameweek']==gameweek),'games played'].round() - all_df.loc[(all_df['gameweek']==gameweek-i),'games played'].round()) )

In [5]:
# top form players
all_df[all_df['gameweek']==gameweek][['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10']].sort_values(by=['form 5'], ascending=False).head(20)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10
id,,,,,,,,
104,Alonso,5.000000,22,4.4,18.226721,3.645344,7.531636,7.531636
79,Alzate,8.928571,25,2.8,23.810274,2.666751,7.348812,3.599380
542,Reguilón,12.926829,53,4.1,49.700672,3.844769,7.273494,4.316170
251,Mané,19.074074,103,5.4,109.718841,5.752250,7.105491,5.622658
437,Antonio,14.000000,63,4.5,64.193156,4.585225,6.758836,5.113454
568,Telles,7.096774,22,3.1,24.928477,3.512649,6.703519,3.500880
24,Saka,18.863636,83,4.4,86.905380,4.607032,6.463808,5.821801
302,Fernandes,21.891892,162,7.4,150.687037,6.883235,6.113249,7.228306
481,Pereira,18.048780,74,4.1,60.794731,3.368357,5.887820,4.455675


In [6]:
# save data
filepath = '../data/fpl/all_data.csv'
all_df.to_csv(filepath)

filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
all_df[all_df['gameweek']==gameweek].to_csv(filepath)